In [7]:
import pandas as pd
import plotly.express as px
import os

In [10]:
# Carregar dados adicionais de JSON, se existir
JSON_PATH = 'dados.json'
if os.path.exists(JSON_PATH):
    with open(JSON_PATH, 'r', encoding='utf-8') as f:
        dados_json = pd.read_json(f)
else:
    dados_json = pd.DataFrame()
    
dados_json['cob'] = 52

dados_json

,data,hora,duracao,fila,holdtime,teleatendente,estado,cob
0,2024-10-17,14:08:44,19,601,27,SIP/2002,1,52
1,2024-10-17,14:07:31,4,601,32,SIP/2002,1,52
2,2024-10-17,14:05:19,0,601,10,None,0,52
3,2024-10-17,14:03:05,243,601,5,SIP/2002,1,52
4,2024-10-17,14:02:30,294,601,4,SIP/2001,1,52
...,...,...,...,...,...,...,...,...
995,2024-10-13,06:41:00,0,601,5,None,0,52
996,2024-10-13,06:38:55,0,601,5,None,0,52
997,2024-10-13,06:35:43,41,601,4,SIP/2001,1,52
998,2024-10-13,06:34:22,42,601,3,SIP/2001,1,52


In [11]:
# Examinar a estrutura dos dados
print("Colunas disponíveis:")
print(dados_json.columns.tolist())
print(f"\nShape dos dados: {dados_json.shape}")
print(f"\nPrimeiras 5 linhas:")
print(dados_json.head())

Colunas disponíveis:
['data', 'hora', 'duracao', 'fila', 'holdtime', 'teleatendente', 'estado', 'cob']

Shape dos dados: (1000, 8)

Primeiras 5 linhas:
         data      hora  duracao  fila  holdtime teleatendente  estado  cob
0  2024-10-17  14:08:44       19   601        27      SIP/2002       1   52
1  2024-10-17  14:07:31        4   601        32      SIP/2002       1   52
2  2024-10-17  14:05:19        0   601        10          None       0   52
3  2024-10-17  14:03:05      243   601         5      SIP/2002       1   52
4  2024-10-17  14:02:30      294   601         4      SIP/2001       1   52


In [12]:
# Converter coluna 'data' para datetime
dados_json['data'] = pd.to_datetime(dados_json['data'])

# Criar coluna datetime combinando data e hora
dados_json['datetime'] = pd.to_datetime(dados_json['data'].dt.strftime('%Y-%m-%d') + ' ' + dados_json['hora'].astype(str))

# Dicionário para mapear os valores de COB para os nomes das regiões
cob_legend = {
    21: '2ºCOB - Uberlândia',
    22: '2ºCOB - Uberaba', 
    31: '3ºCOB - Juiz de Fora',
    32: '3ºCOB - Barbacena',
    4: '4ºCOB - Montes Claros',
    51: '5ºCOB - Governador Valadares',
    52: '5ºCOB - Ipatinga',
    61: '6ºCOB - Varginha'
}

# Mapear COB para nomes
dados_json['cob_nome'] = dados_json['cob'].map(cob_legend)

print(f"Dados preparados: {dados_json.shape}")
print(f"Período dos dados: {dados_json['datetime'].min()} até {dados_json['datetime'].max()}")
print(f"COBs únicos: {dados_json['cob_nome'].unique()}")

Dados preparados: (1000, 10)
Período dos dados: 2024-10-13 06:32:40 até 2024-10-17 14:08:44
COBs únicos: ['5ºCOB - Ipatinga']


In [14]:
# Exemplo de filtros (simulando os filtros do dashboard)
data_inicio = '2024-10-13'
data_fim = '2024-10-17'
hora_inicio = '00:00'
hora_fim = '23:59'
cobs_selecionados = [52]  # Lista de COBs selecionados

# Criar datetime de início e fim para filtro
datetime_inicio = pd.to_datetime(f"{data_inicio} {hora_inicio}")
datetime_fim = pd.to_datetime(f"{data_fim} {hora_fim}")

# Filtrar dados
dados_filtrados = dados_json[
    (dados_json['datetime'] >= datetime_inicio) & 
    (dados_json['datetime'] <= datetime_fim) &
    (dados_json['cob'].isin(cobs_selecionados))
].copy()

print(f"Dados após filtro: {len(dados_filtrados)} registros")

# Agrupar por data/hora e COB para contar chamadas
chamadas_por_datetime_cob = dados_filtrados.groupby(['datetime', 'cob_nome']).size().reset_index(name='quantidade_chamadas')

print(f"Pontos para o gráfico: {len(chamadas_por_datetime_cob)}")
print("Primeiros registros agrupados:")
print(chamadas_por_datetime_cob.head())

# Estrutura do gráfico está pronta para implementação no app.py

Dados após filtro: 1000 registros
Pontos para o gráfico: 1000
Primeiros registros agrupados:
             datetime          cob_nome  quantidade_chamadas
0 2024-10-13 06:32:40  5ºCOB - Ipatinga                    1
1 2024-10-13 06:34:22  5ºCOB - Ipatinga                    1
2 2024-10-13 06:35:43  5ºCOB - Ipatinga                    1
3 2024-10-13 06:38:55  5ºCOB - Ipatinga                    1
4 2024-10-13 06:41:00  5ºCOB - Ipatinga                    1


In [15]:
import numpy as np

# Recarregar dados originais sem o campo COB
dados_originais = pd.read_json('dados.json')

# Lista de COBs disponíveis
cobs_disponiveis = [21, 22, 31, 32, 4, 51, 52, 61]

# Adicionar campo COB com valores aleatórios
np.random.seed(42)  # Para reproduzibilidade
dados_originais['cob'] = np.random.choice(cobs_disponiveis, size=len(dados_originais))

# Verificar a distribuição dos COBs
print("Distribuição dos COBs:")
print(dados_originais['cob'].value_counts().sort_index())

# Salvar de volta no arquivo JSON
dados_originais.to_json('dados.json', orient='records', date_format='iso', indent=2)

print(f"\nArquivo dados.json atualizado com {len(dados_originais)} registros")
print("Primeiras 5 linhas com campo COB:")
print(dados_originais[['data', 'hora', 'cob']].head())

Distribuição dos COBs:
cob
4     125
21    133
22    117
31    113
32    146
51    113
52    119
61    134
Name: count, dtype: int64

Arquivo dados.json atualizado com 1000 registros
Primeiras 5 linhas com campo COB:
         data      hora  cob
0  2024-10-17  14:08:44   52
1  2024-10-17  14:07:31   32
2  2024-10-17  14:05:19    4
3  2024-10-17  14:03:05   52
4  2024-10-17  14:02:30   31


In [16]:
# Verificar se os dados foram salvos corretamente
dados_verificacao = pd.read_json('dados.json')

print(f"Dados carregados: {len(dados_verificacao)} registros")
print(f"Colunas: {list(dados_verificacao.columns)}")
print(f"COBs únicos: {sorted(dados_verificacao['cob'].unique())}")

# Verificar se todos os COBs estão representados no dropdown
cob_legend = {
    21: '2ºCOB - Uberlândia',
    22: '2ºCOB - Uberaba',
    31: '3ºCOB - Juiz de Fora',
    32: '3ºCOB - Barbacena',
    4: '4ºCOB - Montes Claros',
    51: '5ºCOB - Governador Valadares',
    52: '5ºCOB - Ipatinga',
    61: '6ºCOB - Varginha'
}

unique_cob_values = dados_verificacao['cob'].sort_values().unique()
unique_destinos = [{'label': cob_legend.get(cob, f'COB {cob}'), 'value': cob} for cob in unique_cob_values if cob in cob_legend]

print(f"\nOpções do dropdown ({len(unique_destinos)}):")
for destino in unique_destinos:
    print(f"  - {destino['label']} (valor: {destino['value']})")

Dados carregados: 1000 registros
Colunas: ['data', 'hora', 'duracao', 'fila', 'holdtime', 'teleatendente', 'estado', 'cob']
COBs únicos: [np.int64(4), np.int64(21), np.int64(22), np.int64(31), np.int64(32), np.int64(51), np.int64(52), np.int64(61)]

Opções do dropdown (8):
  - 4ºCOB - Montes Claros (valor: 4)
  - 2ºCOB - Uberlândia (valor: 21)
  - 2ºCOB - Uberaba (valor: 22)
  - 3ºCOB - Juiz de Fora (valor: 31)
  - 3ºCOB - Barbacena (valor: 32)
  - 5ºCOB - Governador Valadares (valor: 51)
  - 5ºCOB - Ipatinga (valor: 52)
  - 6ºCOB - Varginha (valor: 61)
